In [1]:
import time
import numpy as np
import pandas as pd
import ringity as rng
import networkx as nx
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from itertools import product
from ipywidgets import widgets
from collections import defaultdict

In [2]:
N = 2**9

In [3]:
def time_estimation(i, iter_len):
    TIME2 = time.time()
    t_cur = TIME2 - TIME1
    prog = (i+1)/iter_len
    t_rem = t_cur * (1/prog - 1)
    print(f"time passed: {t_cur:.2f}sec - progress: {100*prog:.2f}% - time remaining: {t_rem:.2f}sec",
          end = '\r')

In [4]:
scores = defaultdict(list)
n_a_samples = 10
n_b_samples = 10
n_r_samples = 10
n_epochs = 5

a_arr = np.linspace(0.01, 0.5, n_a_samples)
b_arr = np.linspace(0.70, 1.0, n_b_samples)
r_arr = np.linspace(0.01, 0.2, n_r_samples)

TIME1 = time.time()
count = 0
for a, beta, rho in product(a_arr, b_arr, r_arr):
    for epoch in range(n_epochs):
        time_estimation(count, n_a_samples*n_b_samples*n_r_samples*n_epochs)
        count += 1
        try:
            G = rng.network_model(N=N, rho=rho, beta=beta, a=a)
            score = rng.diagram(G).ring_score() # deprication warning
        except (ValueError, rng.DisconnectedGraphError):
            score = np.nan
        
        scores[(a, beta, rho)].append(score)

In [6]:
plot_df = (pd.
            DataFrame(scores).
            melt(var_name = ['a', 'beta', 'rho'],
                 value_name = 'score'))

In [7]:
# plot_df.to_csv("/Users/dottolab/Desktop/ring_scores2.csv")

# Widget

In [8]:
def get_curr_plot_df(plot_df):
    curr_df = (plot_df[(
                    plot_df.
                        beta.
                        between(beta_slider.value - beta_step_size / 2,
                                beta_slider.value + beta_step_size / 2)) 
                       &
                    plot_df.
                        rho.
                        between(rho_slider.value - rho_step_size / 2,
                                rho_slider.value + rho_step_size / 2)].
                 groupby('a').
                 mean().
                 reset_index())
    return curr_df

def response(change):
    tmp_df = get_curr_plot_df(plot_df)
    with g.batch_update():
        g.data[0].x = tmp_df.a
        g.data[0].y = tmp_df.score

In [12]:
beta_step_size = 0.05
rho_step_size = 0.03

beta_slider = widgets.FloatSlider(
                            value = max(b_arr),
                            max = max(b_arr),
                            min = min(b_arr),
                            step = beta_step_size,
                            description = 'beta'
                        )

rho_slider = widgets.FloatSlider(
                            value = max(r_arr),
                            min = min(r_arr),
                            max = max(r_arr),
                            step = rho_step_size,
                            description = 'rho'
                        )

container = widgets.VBox([beta_slider, rho_slider])

curr_plot_df = get_curr_plot_df(plot_df)
trace = go.Scatter(x = curr_plot_df.a, y =  curr_plot_df.score, name = 'ring-score')
g = go.FigureWidget(data = trace,
                    layout = go.Layout(title = {'text' : f'Ring-scores under different parameters for N={N}'})
                   )

g.layout.xaxis.title = 'window size (a)'
g.layout.yaxis.title = 'ring-score'
g.layout.yaxis.range = (-0.05, 1.05)

In [13]:
beta_slider.observe(response, 
                    names = "value")
rho_slider.observe(response, 
                    names = "value")

In [14]:
widgets.VBox([beta_slider, rho_slider, g])

In [84]:
beta_slider.value = 1
rho_slider.value = 0.1

In [85]:
get_curr_plot_df(plot_df)

,a,beta,rho,score
0,0.05,1.0,0.1,NaN
1,0.20,1.0,0.1,0.923813
2,0.35,1.0,0.1,0.577272
3,0.50,1.0,0.1,0.019097


# Spielwiese